In [1]:
import copy
from pathlib import Path
import warnings
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
model_paths = ["Save_File/seperate_model_6hour_05_31/A.ckpt",
                "Save_File/seperate_model_6hour_05_31/B.ckpt",
                "Save_File/seperate_model_6hour_05_31/C.ckpt",
                "Save_File/seperate_model_6hour_05_31/D.ckpt",
                "Save_File/seperate_model_6hour_05_31/E.ckpt"]

# model_paths = ["Save_File/seperate_model_24hour_05_31/A.ckpt",
#                 "Save_File/seperate_model_24hour_05_31/B.ckpt",
#                 "Save_File/seperate_model_24hour_05_31/C.ckpt",
#                 "Save_File/seperate_model_24hour_05_31/D.ckpt",
#                 "Save_File/seperate_model_24hour_05_31/E.ckpt"]

# model_paths = ["Save_File/seperate_model_best_05_28/A/checkpoints/epoch=39-step=1199.ckpt",
#                 "Save_File/seperate_model_best_05_28/B/checkpoints/epoch=57-step=1739.ckpt",
#                 "Save_File/seperate_model_best_05_28/C/checkpoints/epoch=41-step=1259.ckpt",
#                 "Save_File/seperate_model_best_05_28/D/checkpoints/epoch=44-step=1349.ckpt",
#                 "Save_File/seperate_model_best_05_28/E/checkpoints/epoch=40-step=1229.ckpt"]

input_length = 168
output_length = 6

dataframes = []

group_name = ["A", "B", "C", "D", "E"]

for i in range(len(group_name)):
    dataset = pd.read_csv(f"Refined_Data/Grouped_Data/{group_name[i]}.csv", index_col=0)
    dataframes.append(dataset)

In [3]:
group = [
    [34, 40, 42, 41, 4, 10, 11, 12],
    [35, 6, 48, 27, 57, 8, 25, 56, 26, 55, 47, 13, 53, 18, 7, 17, 46],
    [31, 33, 9, 3, 1, 32],
    [29, 38, 43, 58, 15, 22, 39, 54, 23, 44, 45, 37, 52, 2, 14],
    [21, 19, 50, 49, 20, 51, 30, 36, 28, 59, 5, 60, 16, 24]
]

random_choice = 12 # random.randrange(2,13)

start_point = -168 * random_choice - 1
duration = 168

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

all_smape = []
part_smape = [[],[],[],[],[]]

In [4]:
for i in range(len(model_paths)):
    now_dataframe = dataframes[i]
    now_dataframe['num'] = now_dataframe['num'].apply(str)
    now_dataframe['energy_group'] = now_dataframe['energy_group'].apply(str)
    now_dataframe['Week'] = now_dataframe['Week'].apply(str)
    now_dataframe['24Hour'] = now_dataframe['24Hour'].apply(str)
    
    now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

    for building in group[i]:
        now_building_dataframe = now_dataframe.loc[now_dataframe["num"] == str(building)].copy()
        now_building_dataframe = now_building_dataframe[start_point:start_point+input_length+duration]

        original = now_building_dataframe.iloc[0:input_length+output_length]["kWH"].tolist()
        prediction = now_building_dataframe.iloc[0:input_length+output_length]["kWH"].tolist()  

        total_cycle = (len(now_building_dataframe) - input_length - output_length) // output_length

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_building_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_building_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            raw_predictions = now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_building_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_building_dataframe.iloc[now_start_point+input_length+p, now_building_dataframe.columns.get_loc("kWH")] = raw_predictions[p]
            
        # plt.rcParams["figure.figsize"] = (30,10)
        # fig = plt.figure()
        # graph = fig.add_subplot(1, 1, 1)
        # graph.plot(original, color='red')
        # graph.plot(prediction, color='blue')
        # plt.show()

        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        print(smape_loss)
        all_smape.append(smape_loss)
        part_smape[i].append(smape_loss)

        # break
    # break

print("\n")
print(f"selected time slice : {random_choice}")
print(f"totoal score : {np.mean(np.array(all_smape))}")
print(f"A score : {np.mean(np.array(part_smape[0]))}")
print(f"B score : {np.mean(np.array(part_smape[1]))}")
print(f"C score : {np.mean(np.array(part_smape[2]))}")
print(f"D score : {np.mean(np.array(part_smape[3]))}")
print(f"E score : {np.mean(np.array(part_smape[4]))}")

10.431018455066248
8.655864623901136
13.137274888721418
8.336362169783818
21.326386879300546
3.831519751474167
4.1620519670405445
6.2893990705878
10.462290807597455
6.21513134499569
13.394667194045695
16.935773686914242
8.877728063591507
5.323950950163385
8.920937539137121
9.797729316809589
8.608966406926243
5.627761370456362
3.834587057271714
7.675635457672593
14.340592596140691
13.529864870052853
10.92157208819844
7.829063876307937
9.39841196102681
2.1758349562763244
1.7779051316050178
4.520041446212153
1.0766312551763568
0.8438165843528204
0.6604558372126648
8.425258974238119
5.658616776090621
5.149467734115522
4.459954378133337
14.15730279625681
5.985083992807871
7.698031955272783
3.0697955899022986
4.746940061624305
3.763813323875667
13.804147803891999
9.410020936929667
5.303910037402687
2.598428372717446
7.668461863138704
6.210793820231576
9.30661862416872
8.179805355512723
6.179110730179635
6.548176865693888
3.6298452817595805
6.553114102286887
4.131015953668031
5.80139809897473